<a href="https://colab.research.google.com/github/akrity8521/Medicare-Fraud-Detection/blob/main/Code_Thesis_Implementation_catboost_partD(without_autoencoders_and_without_SMOTE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
#importing libraries 
import pandas as pd
import numpy as np
import scipy
import os 
import matplotlib.pyplot as plt
import random
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from scipy.stats import ttest_ind

from sklearn.metrics import brier_score_loss, precision_score, recall_score,f1_score, roc_auc_score, accuracy_score 
from sklearn.metrics import confusion_matrix, roc_curve, classification_report
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_extraction import DictVectorizer
from sklearn.manifold import TSNE
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split


from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense
from sklearn.manifold import TSNE
from keras import regularizers


import catboost as cb
import xgboost as xgb
from sklearn import ensemble 
import lightgbm as lgb

In [ ]:
#connnecting drive with google colab
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#reading part D dataset
dataset = pd.read_csv('/content/drive/My Drive/PartD_Prescriber_PUF_NPI_Drug_18.txt',sep="\t",nrows=1000000)

In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset= dataset.loc[:,['npi', 'specialty_description', 'nppes_provider_state',\
                        'drug_name','bene_count','total_claim_count','total_30_day_fill_count','total_day_supply','total_drug_cost']]

In [ ]:
dataset = dataset.drop_duplicates()

In [ ]:
dataset=dataset.dropna()

In [ ]:
# one-hot encoding
rated_dummies1 = pd.get_dummies(dataset.nppes_provider_state)
dataset=pd.concat([dataset, rated_dummies1], axis=1)
rated_dummies2 = pd.get_dummies(dataset.specialty_description)
dataset=pd.concat([dataset, rated_dummies2], axis=1)
rated_dummies3 = pd.get_dummies(dataset.drug_name) 
dataset=pd.concat([dataset, rated_dummies3], axis=1)
dataset = dataset.drop(['nppes_provider_state','specialty_description','drug_name'], axis=1)

In [ ]:
#reading LEIE dataset
IELErawdata = "/content/drive/My Drive/LEIE.csv"
IELE_pd = pd.read_csv(IELErawdata)

In [ ]:
npifraud_pd0 = IELE_pd.loc[:,['NPI','EXCLTYPE']]
npifraud_pd1 = npifraud_pd0.query('NPI !=0')
rename_dict = {'NPI':'npi', 'EXCLTYPE':'is_fraud'}
npi_fraud_pd = npifraud_pd1.rename(columns=rename_dict)
npi_fraud_pd['is_fraud'] = 1

In [ ]:
#merging the two dataset
Features_pd1 = pd.merge(dataset,npi_fraud_pd, how ='left',on = 'npi')
Features_pd1.fillna(0, inplace=True)
Features_pd1['is_fraud'].value_counts()
Features_pd1[Features_pd1['is_fraud']==1].count()
FeaturesAll_pd=Features_pd1
FeaturesAll_pd.nunique()
FeaturesAll_pd = FeaturesAll_pd.drop(['npi'], axis=1)
FeaturesAll_pd

In [ ]:
x = FeaturesAll_pd.drop(columns=['is_fraud'], axis=1)
y = FeaturesAll_pd['is_fraud']

In [ ]:
#train-test split
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.25, random_state=101, stratify=y)

In [ ]:
#training catboost classifier
clf = cb.CatBoostClassifier()
clf.fit(train_x, train_y)
pred_y = clf.predict(val_x)
print ("")
print ("confusion_matrixt: ")
print (confusion_matrix(val_y, pred_y))

print ("")
print (classification_report(val_y, pred_y))
print ("Precision: ", precision_score(val_y, pred_y))
print ("Recall: ", recall_score(val_y, pred_y))
print ("F1 Score: ", f1_score(val_y, pred_y))
print ("Auc Score: ", roc_auc_score(val_y, pred_y))
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
#training adaboost classifier
clf = ensemble.AdaBoostClassifier()
clf.fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("confusion_matrixt: ")
print (confusion_matrix(val_y, pred_y))

print ("")
print (classification_report(val_y, pred_y))
print ("Precision: ", precision_score(val_y, pred_y))
print ("Recall: ", recall_score(val_y, pred_y))
print ("F1 Score: ", f1_score(val_y, pred_y))
print ("Auc Score: ", roc_auc_score(val_y, pred_y))
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
#training xgboost classifier
clf =  xgb.XGBClassifier()
clf.fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("confusion_matrixt: ")
print (confusion_matrix(val_y, pred_y))

print ("")
print (classification_report(val_y, pred_y))
print ("Precision: ", precision_score(val_y, pred_y))
print ("Recall: ", recall_score(val_y, pred_y))
print ("F1 Score: ", f1_score(val_y, pred_y))
print ("Auc Score: ", roc_auc_score(val_y, pred_y))
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
#training lightGBM classifier
clf =  lgb.LGBMClassifier()
clf.fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("confusion_matrixt: ")
print (confusion_matrix(val_y, pred_y))

print ("")
print (classification_report(val_y, pred_y))
print ("Precision: ", precision_score(val_y, pred_y))
print ("Recall: ", recall_score(val_y, pred_y))
print ("F1 Score: ", f1_score(val_y, pred_y))
print ("Auc Score: ", roc_auc_score(val_y, pred_y))
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))